In [1]:
import math
import re

from tqdm import tqdm

from eval import get_eval_data, pointwise_eval
from utils import summarize

/Users/jaehyungpark/Documents/holiday-mission_박재형/mission2_박재형/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [2]:
print(get_eval_data()[0])

P01: 맞다요즘 싸이월드 새로 생겼다매
P02: 나 싸이월드 복구하고 싶다
P01: 웅 키키귀여우뉴사진
P02: 진짜 추억돋네
P01: 짱많아 옛날 사진들
P01: 맨날 퍼가요~ 이거 했자나
P02: 맞아 싸이월드 미니미
P01: 추억 돋아서 너무조아
P02: 꾸미는거
P01: 마자
P02: 재밌었는데
P01: 귀여워 도토리충전
P02: 네이트온도 하고
P02: 도토리도 환불해준대자나
P01: 마자 키키개웃겨
P01: 요즘 사회적으로 멀 자꾸하나바
P02: 대박이야 진짜
P02: 그니깐 키키
P01: 아니 나오늘 엄마가 뭐 신청해달라그래서
P02: 웅
P01: 지원금 ? 신청함
P01: 진짜 요즘 지원금 엄청 많이 받았어
P02: 와 대박
P01: 뭔지는 잘 모르는데 이것저것 지원해주는거 많아서 좋은듯
P02: 다행이다
P01: 국가제도가 진짜 괜차나
P01: 학교에서도
P02: 두분다
P01: 맨날 장학금 해준다고
P02: 일을 못하는
P01: 이것저것 지원해주자나
P02: 상황이셔서 더
P01: 마자ㅠㅠ
P02: 도움주시고
P01: 나 그래서 진짜 그런거로
P02: 다행이다그래도
P01: 돈 마니받앗어
P02: 와대박


In [5]:
summary = summarize(
    conversation=get_eval_data()[0],
    prompt=PROMPT_BASELINE,
    model='claude-3-haiku-20240307'
)
eval_comment = pointwise_eval(get_eval_data()[0], summary)

print(summary)
print(eval_comment)

1. 두 사람은 과거 싸이월드 사용 경험을 회상하며 추억을 공유하고 있다.
2. 최근 정부와 학교에서 제공하는 다양한 지원금 혜택에 대해 이야기하고 있다.
3. 이러한 지원금 덕분에 경제적으로 어려운 상황에서도 도움을 받고 있다고 말하고 있다.
The AI assistant's response provides a concise summary of the conversation, capturing the main points discussed by the users. It accurately identifies the two main topics: reminiscing about Cyworld and discussing various support funds. However, the response could be improved by offering more specific details or insights into the conversation. Overall, it is helpful and relevant but lacks depth.

Rating: [[7]]


In [6]:
summary = summarize(
    conversation=get_eval_data()[0],
    prompt=PROMPT_BASELINE,
    model='gemini-2.0-flash-exp'
)
eval_comment = pointwise_eval(get_eval_data()[0], summary)

print(summary)
print(eval_comment)

E0000 00:00:1759733322.531326 4113107 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


두 사용자는 새로 나온 싸이월드에 대한 추억을 공유하며, 특히 미니미 꾸미기, 도토리 충전 등에 대한 향수를 느낀다. P01은 최근 정부 및 학교의 다양한 지원금 제도를 통해 경제적 도움을 많이 받았다고 언급한다. P02는 P01의 상황에 공감하며 지원금 혜택에 대해 긍정적으로 반응한다.

The AI assistant's response provides a concise summary of the conversation, capturing the main points discussed by the users, such as their nostalgia for Cyworld and the mention of various support funds. The summary is relevant and accurate, reflecting the key elements of the conversation without adding unnecessary information. However, it could have been slightly more detailed to capture the emotional tone and the flow of the conversation better.

Rating: [[8]]


In [7]:
summary = summarize(
    conversation=get_eval_data()[0],
    prompt=PROMPT_BASELINE,
    model='gpt-3.5-turbo-0125',
)
eval_comment = pointwise_eval(get_eval_data()[0], summary)

print(summary)
print(eval_comment)

두 사용자는 싸이월드를 통해 추억을 공유하고, 최근 받은 지원금에 대해 이야기하며 국가제도에 대한 긍정적인 평가를 나누었습니다.
The AI assistant's response provides a concise summary of the conversation, capturing the main points discussed by the users: reminiscing about Cyworld and discussing recent support funds and positive views on government programs. However, the summary is somewhat generic and lacks depth, failing to capture the nuances and emotional tone of the conversation. It could have been more detailed to better reflect the users' sentiments and specific points mentioned.

Rating: [[6]]


In [ ]:
models = [
    'claude-3-haiku-20240307',
    'gemini-2.0-flash-exp',
    'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(8)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=PROMPT_BASELINE,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

In [ ]:
for model in scores:
    print(scores[model], model)

In [ ]:
for model in scores:
    mean = sum(scores[model]) / len(scores[model])
    variance = sum((x - mean) ** 2 for x in scores[model]) / (len(scores[model]) - 1)
    std_dev = math.sqrt(variance)
    print(f'{model}: {mean} / {round(std_dev, 2)}')

In [5]:
for model in scores:
    print(model, max(scores[model]), min(scores[model]))

NameError: name 'scores' is not defined